# scraping한 rawdata 전처리 - info(performance)
- 형식에 맞게 클리닝

In [205]:
import pandas as pd

## info 데이터 전처리 (완료)
1. 모든 뮤지컬, 연극 하나의 파일로 합치기
2. 클리닝
    - location: 장소 ~ (자세히) 없애기
    - performance_time: 공연기간\n 없애기
    - period: 공연시간\n ~ 분 없애기
    - age_requirement: 관람연령\n 없애기

In [206]:
import os

def concat_info_data():
    folder_path = "../data/rawdata/"
    infos = []

    for file_path in os.listdir(folder_path):
        if "info" in file_path:
            info = pd.read_csv(folder_path + file_path)
            infos.append(info)

    df = pd.concat(infos, ignore_index=True)
    df.to_csv(f"../data/preprocessed_data/total_info.csv", index=False, encoding='utf-8')

    return df

In [207]:
info = concat_info_data()
info

,performance_id,title,performance_type,location,period,performance_time,age_requirement
0,23008837,레베카,뮤지컬,장소블루스퀘어 신한카드홀(자세히),공연기간\n2023.08.19 ~2023.11.19,공연시간\n175분(인터미션 20분 포함),관람연령\n8세이상 관람가능
1,24001487,브론테,뮤지컬,장소링크아트센터드림 드림1관(자세히),공연기간\n2024.03.04 ~2024.06.02,공연시간\n100분,관람연령\n초등학생이상 관람가
2,23001653#,영웅,뮤지컬,장소블루스퀘어 신한카드홀(자세히),공연기간\n2023.03.17 ~2023.05.21,공연시간\n160분(인터미션 20분 포함),관람연령\n8세이상 관람가능
3,24001224,오즈,뮤지컬,장소대학로 TOM(티오엠) 2관(자세히),공연기간\n2024.02.27 ~2024.05.06,공연시간\n100분,관람연령\n8세이상 관람가능
4,24002031,웨스턴 스토리,뮤지컬,장소인터파크 유니플렉스 1관(자세히),공연기간\n2024.03.13 ~2024.06.09,공연시간\n145분(인터미션 15분 포함),관람연령\n14세 이상 관람가
5,22001159,늘근도둑이야기,연극,장소대학로 아트포레스트 1관(자세히),공연기간\n2022.03.05 ~2024.05.31,공연시간\n100분,관람연령\n12세이상
6,24000171,라면,연극,장소대학로 해피씨어터(자세히),공연기간\n2024.02.01 ~2024.05.31,공연시간\n100분,관람연령\n만 11세이상
7,19018229#,쉬어매드니스,연극,장소콘텐츠박스(KONTENTZ BOX)(자세히),공연기간\n2015.11.12 ~오픈런,공연시간\n110분(인터미션 7분 포함),관람연령\n만 15세이상
8,23008491,죽여주는 이야기,연극,장소지인시어터(구.알과핵소극장)(자세히),공연기간\n2023.07.01 ~오픈런,공연시간\n100분,관람연령\n8세이상 관람가능
9,22014277#,한뼘사이,연극,장소라온아트홀(자세히),공연기간\n2017.03.01 ~오픈런,공연시간\n100분,관람연령\n만 12세이상


In [208]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   performance_id    10 non-null     object
 1   title             10 non-null     object
 2   performance_type  10 non-null     object
 3   location          10 non-null     object
 4   period            10 non-null     object
 5   performance_time  10 non-null     object
 6   age_requirement   10 non-null     object
dtypes: object(7)
memory usage: 688.0+ bytes


- location: 장소 ~ (자세히) 없애기
- performance_time: 공연기간\n 없애기
- period: 공연시간\n ~ 분 없애기
- age_requirement: 관람연령\n 없애기

In [209]:
# location 클리닝
info["location"] = info["location"].apply(lambda x: x[2:-5])

# performance_time 클리닝
info["performance_time"] = info["performance_time"].apply(lambda x: x.split("\n")[1])

# period 클리닝
info["period"] = info["period"].apply(lambda x: x.split("\n")[1])

# age_requirement 클리닝
info["age_requirement"] = info["age_requirement"].apply(lambda x: x.split("\n")[1])

- 공연 기간 start_date, end_date 로 분할 및 period 컬럼 삭제

In [210]:
info[['start_date', 'end_date']] = info['period'].str.split(' ~', expand=True)
info.drop("period", axis=1, inplace=True)

- age_requirement 관람 연령 형식 통일
    - n세 이상, ~ 이상, 만 n세 이상
    - 전부 기본 형태는 ~ 이상 ~ 이지만 그 사이 띄어쓰기 등 형식이 다르다

In [211]:
# "이상" 으로 split 후 .strip, "이상"으로 join
info["age_requirement"] = info["age_requirement"].apply(lambda x: x.split("이상")[0].strip() + " 이상")

- column 이름 (field 이름) 통일

In [213]:
info.columns

Index(['performance_id', 'title', 'performance_type', 'location',
       'performance_time', 'age_requirement', 'start_date', 'end_date'],
      dtype='object')

In [212]:
# info.rename(columns={})

,performance_id,title,performance_type,location,performance_time,age_requirement,start_date,end_date
0,23008837,레베카,뮤지컬,블루스퀘어 신한카드홀,175분(인터미션 20분 포함),8세 이상,2023.08.19,2023.11.19
1,24001487,브론테,뮤지컬,링크아트센터드림 드림1관,100분,초등학생 이상,2024.03.04,2024.06.02
2,23001653#,영웅,뮤지컬,블루스퀘어 신한카드홀,160분(인터미션 20분 포함),8세 이상,2023.03.17,2023.05.21
3,24001224,오즈,뮤지컬,대학로 TOM(티오엠) 2관,100분,8세 이상,2024.02.27,2024.05.06
4,24002031,웨스턴 스토리,뮤지컬,인터파크 유니플렉스 1관,145분(인터미션 15분 포함),14세 이상,2024.03.13,2024.06.09
5,22001159,늘근도둑이야기,연극,대학로 아트포레스트 1관,100분,12세 이상,2022.03.05,2024.05.31
6,24000171,라면,연극,대학로 해피씨어터,100분,만 11세 이상,2024.02.01,2024.05.31
7,19018229#,쉬어매드니스,연극,콘텐츠박스(KONTENTZ BOX),110분(인터미션 7분 포함),만 15세 이상,2015.11.12,오픈런
8,23008491,죽여주는 이야기,연극,지인시어터(구.알과핵소극장),100분,8세 이상,2023.07.01,오픈런
9,22014277#,한뼘사이,연극,라온아트홀,100분,만 12세 이상,2017.03.01,오픈런


In [214]:
# csv로 저장
info.to_csv(f"../data/preprocessed_data/total_info_cleaning.csv", index=False, encoding='utf-8')